In [1]:
import json

import torch
from torch import nn
from torch.utils.data import Dataset

import numpy as np
import pandas as pd
import matplotlib.image as mpimg

import loader as L

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
batch_size = 32

In [3]:
filter = ['wildHair', 'cigarette']
# filter = ['male', 'female', 'cigarette']

train_dataset = L.PunksDataset(filter, train=True)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=False
)

test_dataset = L.PunksDataset(filter, train=False)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=True
)

In [4]:
# Explore Dataset
punk, labels = train_dataset[42]
print(punk.shape)
print("filter:", labels)
L.write_image(punk, "42.jpg")


# Explore Loader
punk_batch, labels_batch = next(iter(train_loader))
print(type(punk_batch))
print(type(punk_batch[0]))
print(type(punk_batch[0][0]))
print(type(punk_batch[0][0][0]))
# print(type(punk_batch[0][0][0][0]))
print(punk_batch.shape)
print(punk_batch[0].shape)
print("filter:", labels_batch[0].numpy())


# Write new image with loaded data
L.write_image(punk_batch[0].numpy(), "0.jpg")

(24, 24, 4)
filter: [1 1]
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
torch.Size([32, 24, 24, 4])
torch.Size([24, 24, 4])
filter: [0 0]


In [5]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(24*24*4, 200),
    nn.ReLU(),
    nn.Linear(200, 150),
    nn.ReLU(),
    nn.Linear(150, len(filter)),
    nn.ReLU(),
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

model.to(device)


Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=2304, out_features=200, bias=True)
  (2): ReLU()
  (3): Linear(in_features=200, out_features=150, bias=True)
  (4): ReLU()
  (5): Linear(in_features=150, out_features=2, bias=True)
  (6): ReLU()
)

In [6]:
num_epoch = 10
training_loss = 0
test_loss = 0


for epoch in range(num_epoch):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.type(torch.float64))
        loss.backward()
        optimizer.step()
        training_loss += loss.item()

    print('====> Epoch[{}/{}], loss: {:.6f}'.format(
        epoch + 1, num_epoch, loss.data.item()
    ))

    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            test_loss += criterion(outputs, labels.type(torch.float64))

        test_loss /= len(test_loader.dataset)
        print('====> Test[{}/{}, loss: {:.6f}'.format(
            epoch + 1, num_epoch, test_loss.data.item()
        ))


====> Epoch[1/10], loss: 0.003139
====> Test[1/10, loss: 0.000099
====> Epoch[2/10], loss: 0.001071
====> Test[2/10, loss: 0.000036
====> Epoch[3/10], loss: 0.001822
====> Test[3/10, loss: 0.000070
====> Epoch[4/10], loss: 0.000562
====> Test[4/10, loss: 0.000019
====> Epoch[5/10], loss: 0.000411
====> Test[5/10, loss: 0.000016
====> Epoch[6/10], loss: 0.000993
====> Test[6/10, loss: 0.000041
====> Epoch[7/10], loss: 0.000273
====> Test[7/10, loss: 0.000009
====> Epoch[8/10], loss: 0.000222
====> Test[8/10, loss: 0.000008
====> Epoch[9/10], loss: 0.000890
====> Test[9/10, loss: 0.000037
====> Epoch[10/10], loss: 0.000210
====> Test[10/10, loss: 0.000007
